### Menoplan Fraud Detection Protocol

In [615]:
import pandas as pd # Pandas is an open source library providing data structures and data analysis tools for Python
import re #regex
from functools import reduce 
import numpy as np
from math import isnan
import matplotlib.pyplot as plt
import urllib.request
import json
from datetime import date
from datetime import datetime

In [646]:
beta_file_loc =  "C:/Users/huixin/OneDrive - UC San Diego/menoplan/pilot_phase/merged_analysis/"
pilot_file_loc = "C:/Users/huixin/OneDrive - UC San Diego/menoplan/trial_phase/"
qualtrics_data = pd.read_csv(pilot_file_loc + "qualtrics_data/12jan/12jan_p2.csv")
screening = pd.read_csv(pilot_file_loc +"qualtrics_data/12jan/12jan_p1.csv")
screening=screening.iloc[2:] #drop first two rows
qualtrics_data=qualtrics_data.iloc[2:] #drop first two rows

#helper functions
def straightLining(condDf,  rowNum, colNames): #check if answers to questions in a row are the same
    elems = list((condDf[colNames].eq(condDf[colNames].iloc[rowNum:rowNum+1, 0], axis=0).values)[0])
    return elems.count(elems[0]) == len(elems) # are all elements in the list same?

def replaceEmptyWithNAN(varname, dfName): #replace empty string with NAN and convert col to numeric
    dfName[varname].replace(" ", np.NaN, regex=True)
    dfName[varname]=pd.to_numeric(dfName[varname])

req = urllib.request.Request('https://raw.githubusercontent.com/millbj92/US-Zip-Codes-JSON/master/USCities.json')
with urllib.request.urlopen(req) as response: #master file for matching zipcode to 
    js = json.load(response) 
    
def findState(zc):
    for dct in js:
        if dct.get("zip_code") == zc:
            return dct.get('state')
        
def containQuoteMarks(string):
    if string.startswith("\"") and string.endswith("\""):
        return True 
def getLsLen(string):
    li = list(string.split(","))
    return len(li)

# #data cleaning
ga_data = pd.read_csv(beta_file_loc + "02nov_merged_pageviews_events_os.csv")
qualtrics_data.columns = qualtrics_data.columns.str.replace('Infromation', 'Information')
qualtrics_data.loc[qualtrics_data['C-Timing_First Click'].isna() , 'Cond'] = 'Exp' #create new col to indicate condition
qualtrics_data.loc[~qualtrics_data['C-Timing_First Click'].isna() , 'Cond'] = 'Ctrl'
qualtrics_data = qualtrics_data.rename(columns= #rename cols
                                       {'RecordedDate': 'EndVisitTs',
                                        'Duration (in seconds)': 'durationMins',
                                       'Duration (in seconds)': 'durationMins'}) 
print('Sanity check of column names')
print(list(qualtrics_data.columns))
qualtrics_data['RecipientEmail']

Sanity check of column names
['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'durationMins', 'Finished', 'EndVisitTs', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage', 'Q_RecaptchaScore', 'Q_RelevantIDDuplicate', 'Q_RelevantIDDuplicateScore', 'Q_RelevantIDFraudScore', 'Q_RelevantIDLastStartDate', 'TypeWord', 'LinkSource', 'LinkSourceOpen', 'Educ', 'Ethnicity', 'Randomization_1', 'Randomization_2', 'C-Timing_First Click', 'C-Timing_Last Click', 'C-Timing_Page Submit', 'C-Timing_Click Count', 'C-Website1', 'C-OtherWebsite1', 'C-Website2', 'C-OtherWebsite2', 'C-Peri_Meno_Experie', 'C-Meno_Stage', 'C-Meno_Stage_Trans', 'C-Period_Stop', 'C1.HF', 'C2.NS', 'C3.S', 'C4.BDL', 'C5.FA/N', 'C6.EPM', 'C7.AL', 'C8.FDDB', 'C9.BIWO', 'C10.FWBA', 'C11.F/G', 'C12.AMJ', 'C13.FTW', 'C14.DS', 'C15.ABH', 'C16.DPS', 'C17.DS', 'C18.FLE', 'C19.DS', 'C20.WG', 'C21.IFH', 'C22

2       smeans082173@gmail.com
3            15683tb@gmail.com
4           irutto96@gmail.com
5      Sagalasandra0@gmail.com
6       walkerbravin@gmail.com
7          jeninne72@gmail.com
8         konseilias@gmail.com
9     brandybohannon@gmail.com
10      wegnawegener@gmail.com
Name: RecipientEmail, dtype: object

In [647]:
#convert to numeric, replace ' ' with NAN
ls = ['C-Gender', 'E-gender', 'Q_RecaptchaScore','Randomization_2','Randomization_1', 'C-ATT-CHECK1', 
      'C-ATT-CHECK2', 'E-ATT-CHECK1', 'E-ATT-CHECK2',
    #  'C-Age', 'E-Age',  use birthdate var instead
      'durationMins'] #convert to numeric
for x in ls:
    replaceEmptyWithNAN(x, qualtrics_data)
for x in ['Q_RelevantIDDuplicateScore','Q_RelevantIDFraudScore' ]:
    replaceEmptyWithNAN(x, screening)
     
qualtrics_data['durationMins'] = qualtrics_data['durationMins']/60 #convert duration from secs to mins
first_column =qualtrics_data.pop('Cond')# shift column 'Cond' to first position
qualtrics_data.insert(0, 'Cond', first_column)  
qualtrics_data = qualtrics_data.add_suffix('_p2') #add _p2 to each col to indicate this is part 2 of survey

### Inattention
1. Unusually short period of time exploring the study websites (&lt; 10 minutes) `p1p2df['StartDate_p2']- p1p2['EndDate']`  
    - Check the timing of when they start answering this question. and when they first entered the survey: Please think about your experience with the information on the website. How much do you agree or disagree with each statement?
2. Unusually short survey completion times (&lt; 2 minutes)  `['durationMins']`
3. Selection of all items in multiple response questions 
    - find variable names of multiple-response questions
    - check if all answers were selected  `selectAll`
4. Selection of same response to all items in a list of possible items (straight lining) 
    - find variable names of multiple-response questions 
        - Check if same `straightLining
5. Failure to answer attention check correctly.`['Q_RecaptchaScore'] >= 80`

#### Duplicate or unusual responses to open-ended items
1. Several responses follow the same pattern in terms of phrasing or format  
    - check if open ended answers are duplicated
2. Nonsensical or non-matching response to open ended questions 
    - check manually
    
#### Other inconsistent or unusual responses
1. High level of nonresponse to survey questions or refusal, >10% - Done 2. Mismatched survey answers (for example 2 different menopause stages, presence/absence of menopause symptoms)  - Done
3. Age out of range (&lt;40, &gt; 55) - Done
4. Male sex - Done
5. Indicated wrong study source - TBA
6. Look to see if strange links are sites that direct people to make money 
    - check traffic from GA for Menoplan Website
7. Duplicate open-ended responses - Done

#### Inconsistent responses to verifiable items
1. Suspicious email address (for example at least 10 random numbers or letters in a row or strange punctuation or capitalization) `['?-Email_p2'].str.isupper()`

2. Time of submission (for example, 12-4 am)
3. Business vs personal address - Manual Check, Spokeo
4. Address not real - Manual Check, Spokeo
5. Address includes quotations marks - Done
6. Name/suffix suspicious - Manual Check, Spokeo
7. Business vs. personal phone number - Manual Check, Spokeo
8. Entered zipcode and state mismatch - Done
9. IP address not in the U.S. - Manual Check, Spokeo 
10. IP address appears on more than one entry 

In [648]:
conDf = qualtrics_data[list(qualtrics_data.iloc[:,:185].columns)+list(qualtrics_data.iloc[:, -5:])] #subset only columns relevant to control condition (C-)
expDf = qualtrics_data[list(qualtrics_data.iloc[:, :30].columns) +  list(qualtrics_data.iloc[:, 185:].columns)]
#subset only columns relevant to experimental condition (begin with E-)
#list(expDf.columns)

In [649]:
conDf =conDf[conDf['Cond_p2'] == 'Ctrl'] #subset only  rows from control condition participants
expDf =expDf[expDf['Cond_p2'] == 'Exp'] #subset only rows from experiment condition participants

####  Screening Survey
In addition to procedure above, we can also use the relevantID feature from qualtrics, we can find out the likeliness a survey is duplicated.
- https://www.qualtrics.com/support/survey-platform/survey-module/survey-checker/fraud-detection/

In [650]:
cDuplicate=screening['Q_RelevantIDDuplicateScore'] > 75 
cBot=screening['Q_RelevantIDFraudScore'] > 30
screening[cDuplicate] #which responses are duplicated?
screening[cBot] #which responses are bots?

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,BrowserInfo_Operating System,BrowserInfo_Resolution,Email,Email2,QCity_ExportTag,QState_ExportTag,Referer,cityName,Q_URL,Q_BallotBoxStuffing
14,2022-01-12 21:05:44,2022-01-12 21:06:35,0,37.120.219.106,100,51,1,2022-01-12 21:06:35,R_3oRz5NokXhRAOWQ,NaN,...,Android 11,412x892,NaN,NaN,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
15,2022-01-12 21:07:39,2022-01-12 21:09:13,0,37.120.219.106,100,94,1,2022-01-12 21:09:13,R_2wS9G9mzKXPfeDF,NaN,...,Android 11,412x892,irutto96@gmail.com,irutto96@gmail.com,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
16,2022-01-12 21:20:41,2022-01-12 21:23:51,0,94.140.8.34,100,190,1,2022-01-12 21:23:52,R_2xy3rIrM70kwCdP,NaN,...,Android 11,412x915,aminlostie@gmail.com,aminlostie@gmail.com,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
21,2022-01-12 21:58:55,2022-01-12 22:03:11,0,91.219.212.88,100,256,1,2022-01-12 22:03:12,R_3PXFn9KaglaaHFz,NaN,...,Android 10,360x800,konseilias@gmail.com,konseilias@gmail.com,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
22,2022-01-12 22:03:19,2022-01-12 22:04:29,0,141.94.222.131,100,70,1,2022-01-12 22:04:30,R_1in9IQX5QPHHDp0,NaN,...,Android 11,393x851,NaN,NaN,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
23,2022-01-12 22:04:00,2022-01-12 22:06:00,0,148.72.164.183,100,120,1,2022-01-12 22:06:00,R_3ly585pytpaYfzW,NaN,...,Android 10,393x851,walkerbravin@gmail.com,walkerbravin@gmail.com,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
24,2022-01-12 22:06:33,2022-01-12 22:07:30,0,51.195.252.224,100,57,1,2022-01-12 22:07:30,R_3F56OnyeyVxY6zx,NaN,...,Android 11,393x851,NaN,NaN,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
25,2022-01-12 22:10:18,2022-01-12 22:11:09,0,135.148.149.73,100,51,1,2022-01-12 22:11:09,R_3Ld9GdYiEZYcFTy,NaN,...,Android 11,393x851,NaN,NaN,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
29,2022-01-12 22:32:56,2022-01-12 22:34:44,0,69.167.43.24,100,107,1,2022-01-12 22:34:44,R_1Ka6sTqCh27T07A,NaN,...,Android 11,393x873,NaN,NaN,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
30,2022-01-12 22:38:51,2022-01-12 22:52:12,0,185.202.220.74,100,801,1,2022-01-12 22:52:12,R_26i44mIpvDSVMke,NaN,...,Android 10,360x820,wegnawegener@gmail.com,wegnawegener@gmail.com,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN


In [651]:
#create 'selectAll' col that checks if participant selects all options on a multi-choice question
expDf['selectAll']=expDf["E-Peri_Meno_Exper_p2"].apply(lambda x: (getLsLen(x)))
# check if the zipcode entered matches the state entered (experimental condition)
expDf['zipcodeState'] = expDf['E-Information_9_p2'].astype('int') .apply(lambda x: findState(x))
expDf['E-Information_8_p2'] = expDf['E-Information_8_p2'].apply(lambda x: str(x)).apply(lambda x: x.upper())
expDf['zipcodeX_p2']=expDf['E-Information_8_p2'].isin(expDf.zipcodeState) 
#calculate percentage of NAs for each condition
expDf['percentNA_p2'] = expDf.iloc[:,27:].isna().sum(axis = 1)/len(expDf.iloc[:,27:].columns)
#check if street address has quotation marks around
expDf['QuoteMarks']= expDf['E-Information_5_p2'].apply(lambda x: containQuoteMarks(x))

#now we can begin to run the fraud detection procedure 
recaptcha = expDf['Q_RecaptchaScore_p2'] < 0.8 #recaptcha accuracy less than 80%
cond1=expDf["E-Peri_Meno_Exper_p2"].str.contains(',11') #select other options even though selected 'I refuse to answer'
cond2=expDf["E-ATT-CHECK1_p2"] != 1 #Did not select 'Strongly Agree'
cond3=expDf["E-ATT-CHECK2_p2"] != 5 #Did not select 'Strongly Disagree'
cond4=expDf['E-Web_Tools_p2'].str.contains(',14')#select other options even though selected 'I refuse to answer'

cond6 = ~expDf['Randomization_2_p2'].isin(list(np.arange(61, 81, 1)))  #age out of range
cond7 = expDf['E-gender_p2'] != 1 #not female
cond8=expDf.duplicated(['IPAddress_p2']) #IP address duplicated
cond9=expDf['durationMins_p2'] <=2 #finished survey in less than 2 mins
#cond10 = expDf['zipcodeX_p2'] != True #entered zipcode's state does not match entered state
cond11=expDf['RecipientEmail_p2'].str.isupper()
cond12 = expDf['selectAll'] == 11
# expDf[recaptcha |cond1 | cond2 | cond3 | cond4 | 
#       cond6| #was the age variable removed?
#       cond7| cond8| cond9| cond10|cond11 |cond12]
expDf[cond6]
for x in [recaptcha , cond1, cond2 , cond3 , cond4 , cond6, cond7, cond8,cond9, cond11, cond12]:
    if (x.any()==True):
           print(x.name)
           display(expDf[x][x.name])

Q_RecaptchaScore_p2


8    0.7
Name: Q_RecaptchaScore_p2, dtype: float64

E-ATT-CHECK1_p2


4     5.0
10    4.0
Name: E-ATT-CHECK1_p2, dtype: float64

E-ATT-CHECK2_p2


4     4.0
10    3.0
Name: E-ATT-CHECK2_p2, dtype: float64

In [652]:
##### create 'selectAll' col that checks if participant selects all options on a multi-choice question
conDf['selectAll']=conDf["C-Peri_Meno_Experie_p2"].apply(lambda x: (getLsLen(x)))
# check if zipcode entered matches the state entered (control condition)
conDf['C-Information_9_p2']= (conDf['C-Information_9_p2'].str[:5])
conDf['zipcodeState'] = conDf['C-Information_9_p2'].astype('int') .apply(lambda x: findState(x))
conDf['C-Information_8_p2'] = conDf['C-Information_8_p2'].apply(lambda x: str(x)).apply(lambda x: x.upper())
conDf['zipcodeX_p2']=conDf['C-Information_8_p2'].isin(conDf.zipcodeState)
#calculate percentage of NAs for each condition
conDf['percentNA_p2'] = conDf.iloc[:,27:].isna().sum(axis = 1)/len(conDf.iloc[:,27:].columns)
recaptcha = conDf['Q_RecaptchaScore_p2'] < 0.8 #recaptcha accuracy less than 80%

cond1=conDf["C-Peri_Meno_Experie_p2"].str.contains(',11') #select other options even though selected 'I refuse to answer'
cond2=conDf["C-ATT-CHECK1_p2"] != 1 #Did not select 'Strongly Agree'
cond3=conDf["C-ATT-CHECK2_p2"] != 5 #Did not select 'Strongly Disagree'
#cond5=conDf['C36.ATT CHECK_p2'].isna() #Did not select yes
cond6 = ~conDf['Randomization_2_p2'].isin(list(np.arange(67, 82, 1))) #age out of range
cond7 = conDf['C-Gender_p2'] != 1 #not female
cond8=conDf.duplicated(['IPAddress_p2']) #IP address duplicated
cond9=conDf['durationMins_p2'] <=2 #finished survey in less than 2 mins
#cond10 = conDf['zipcodeX_p2'] != True #entered zipcode's state does not match entered state
cond4=conDf['RecipientEmail_p2'].str.isupper() #all Caps, was the email variable removed?
cond11 = conDf['selectAll'] == 11
#conDf[re]
# conDf[recaptcha  | cond2 | cond3 | cond4 | #cond5| 
#       cond6| 
#       cond7| cond8| cond9| cond10| cond11]
for x in [recaptcha  , cond2 , cond3 , cond4 , cond6, cond7, cond8,cond9, cond11]:
    if (x.any()==True):
        print(x.name)
        display(conDf[x][x.name])
        


C-ATT-CHECK1_p2


6    5.0
Name: C-ATT-CHECK1_p2, dtype: float64

C-ATT-CHECK2_p2


6    4.0
Name: C-ATT-CHECK2_p2, dtype: float64

Randomization_2_p2


5    85
Name: Randomization_2_p2, dtype: int64

C-Gender_p2


6    2.0
Name: C-Gender_p2, dtype: float64

#### How long does it take for participant to move from first o second survey?
We need to merge the first and second survey using `IPAddress` var to calculate the duration taken to complete the survey. Rename var `IPAddress_p2` to `IPAddress` so that they are mergeable. The merged df is called `p1p2df`. 

In [623]:
screening['EndDate']=pd.to_datetime(screening['EndDate'])#when did they finished survey 1
qualtrics_data['StartDate_p2']=pd.to_datetime(qualtrics_data['StartDate_p2'])#when did they start survey 2
qualtrics_data['EndDate_p2']=pd.to_datetime(qualtrics_data['EndDate_p2'])
#rename column from p2 survey to match columns on IP Address
qualtrics_data=qualtrics_data.rename(columns= {'IPAddress_p2':'IPAddress'})
p1p2df = pd.merge(qualtrics_data, screening, on='IPAddress')
#display(p1p2df['StartDate_p2']-p1p2df['EndDate']) # Duration 
#(p1p2df['EndDate_p2']-p1p2df['StartDate_p2']).dt.total_seconds()/60 

p1p2df['durationMins_p2'] > 20



0     False
1      True
2      True
3      True
4      True
5     False
6     False
7      True
8      True
9      True
10     True
Name: durationMins_p2, dtype: bool

#### Does participant of n-th row select the same answer over multiple questions?

In [624]:
attnChck1 = ['E1-Literacy_Skills_p2',
 'E2-Literacy_Skills_p2',
 'E3-Literacy_Skills_p2',
 'E4-Literacy_Skills_p2',
 'E5-Literacy_Skills_p2',
 'E6-Literacy_Skills_p2',
 'E7-Literacy_Skills_p2',
 'E8-Literacy_Skills_p2']
attnChck2 = [ 'E1-Perceived_Quality_p2',
 'E2-Perceived_Quality_p2',
 'E3-Perceived_Quality_p2',
 'E4-Perceived_Quality_p2',
 'E5-Perceived_Quality_p2',
 'E6-Perceived_Quality_p2',
 'E7-Perceived_Quality_p2']
attnChcks = attnChck2 + attnChck1
#straightLining(expDf, 0, attnChcks)
#expDf
attnChcks = [x.replace('E', 'C') for x in attnChcks]
straightLining(conDf, 3, attnChcks)

True

In [625]:
p1p2df

,Cond_p2,StartDate_p2,EndDate_p2,Status_p2,IPAddress,Progress_p2,durationMins_p2,Finished_p2,EndVisitTs_p2,ResponseId_p2,...,BrowserInfo_Operating System,BrowserInfo_Resolution,Email,Email2,QCity_ExportTag,QState_ExportTag,Referer,cityName,Q_URL,Q_BallotBoxStuffing
0,Ctrl,2022-01-12 15:48:31,2022-01-12 16:04:44,0,68.38.9.234,100,16.200000,1,2022-01-12 16:04:44,R_R3uiZFMQeezK4Vj,...,Android 11,320x694,smeans082173@gmail.com,smeans082173@gmail.com,NaN,NaN,https://lm.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
1,Ctrl,2022-01-12 18:27:32,2022-01-12 19:13:20,0,71.166.115.162,100,45.783333,1,2022-01-12 19:13:20,R_3LdMzxjrpQQ5tI1,...,Windows NT 10.0,1600x900,15683tb@gmail.com,15683tb@gmail.com,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
2,Exp,2022-01-12 21:10:27,2022-01-12 21:43:50,0,37.120.219.106,100,33.383333,1,2022-01-12 21:43:51,R_1rIDsQs6LGDFJfk,...,Android 11,412x892,NaN,NaN,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
3,Exp,2022-01-12 21:10:27,2022-01-12 21:43:50,0,37.120.219.106,100,33.383333,1,2022-01-12 21:43:51,R_1rIDsQs6LGDFJfk,...,Android 11,412x892,NaN,NaN,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
4,Exp,2022-01-12 21:10:27,2022-01-12 21:43:50,0,37.120.219.106,100,33.383333,1,2022-01-12 21:43:51,R_1rIDsQs6LGDFJfk,...,Android 11,412x892,irutto96@gmail.com,irutto96@gmail.com,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
5,Ctrl,2022-01-12 21:48:30,2022-01-12 22:02:26,0,105.163.2.178,100,13.933333,1,2022-01-12 22:02:26,R_1rC8WDNP20L8ZVZ,...,Android 11,360x800,Sagalasandra0@gmail.com,Sagalasandra0@gmail.com,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
6,Ctrl,2022-01-12 22:07:11,2022-01-12 22:23:46,0,148.72.164.183,100,16.566667,1,2022-01-12 22:23:46,R_30nH2nDpyHzEFvy,...,Android 10,393x851,walkerbravin@gmail.com,walkerbravin@gmail.com,NaN,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
7,Ctrl,2022-01-12 22:16:58,2022-01-12 22:49:04,0,107.115.21.42,100,32.083333,1,2022-01-12 22:49:04,R_1dsmdo6APFWJeiG,...,Android 10,320x640,jeninne72@gmail.com,jeninne72@gmail.com,NaN,NaN,https://lm.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
8,Exp,2022-01-12 22:04:21,2022-01-12 22:55:52,0,91.219.212.88,100,51.500000,1,2022-01-12 22:55:52,R_yUSjn1u0Z8F3OVz,...,Android 10,360x800,konseilias@gmail.com,konseilias@gmail.com,NaN,NaN,https://l.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN
9,Exp,2022-01-12 22:32:55,2022-01-12 23:11:20,0,98.46.105.190,100,38.400000,1,2022-01-12 23:11:20,R_2a8oPfGTbGqJkYa,...,Android 8.0.0,360x640,brandybohannon@gmail.com,brandybohannon@gmail.com,NaN,NaN,https://lm.facebook.com/,NaN,https://uconn.co1.qualtrics.com/jfe/form/SV_0j...,NaN


In [654]:
conCols = list(conDf.columns[30:-4])
expCols = list(expDf.columns[30:-10])

In [655]:
varE = []
for x in expCols:
    if x.startswith('E'):
        s = x.replace('E', 'I', 1)
        varE.append(s)

In [656]:
varC = []
for x in conCols:
    if x.startswith('C'):
        s = x.replace('C', 'I', 1)
        varC.append(s)

In [629]:
#set(varC) - set(varE)
# {'I-Email_Link_p2', #NonRel
#  'I-Gender_p2',
#  'I-Job Status_p2',
#  'I-Meno_Stage_Trans_p2',
#  'I-Meno_Stage_p2',
#  'I-OtherWebsite1_p2', #NonRel
#  'I-OtherWebsite2_p2', #NonRel
#  'I-Peri_Meno_Experie_p2',
#  'I-Website1_p2', #NonRel
#  'I-Website2_p2', #NonRel
#  'I37.UI_p2'}

In [657]:
set(varE) - set(varC) # from exp

{'I-Job_Status_p2',
 'I-Meno_Stage_Tran_p2',
 'I-Menopause_Stage_p2',
 'I-Peri_Meno_Exper_p2',
 'I-Web_Tools_p2',
 'I-Website_Link_p2',
 'I-gender_p2',
 'I3.1-Useability_p2',
 'I37.UTI_p2'}

In [658]:
newColNameE = list(expDf.columns[:30]) + varE+  list(expDf.columns[-10:])
expDf=expDf.iloc[:, :].set_axis([newColNameE], axis=1)
expDf=expDf.rename(columns= #rename cols
                                       {'I-gender_p2': 'I-Gender_p2',
                                        'I37.UTI_p2': 'I37.UI_p2',
                                       'I-Job_Status_p2': 'I-Job Status_p2',
                                       'I-Peri_Meno_Exper_p2':'I-Peri_Meno_Experie_p2',
                                       'I-Meno_Stage_Tran_p2': 'I-Meno_Stage_Trans_p2',
                                       'I-Menopause_Stage_p2':'I-Meno_Stage_p2'}) 


In [660]:
newColNameC = list(conDf.columns[:30]) + varC+  list(conDf.columns[-9:])
conDf=conDf.iloc[:, :].set_axis([newColNameC], axis=1)

In [677]:
pd.concat([expDf, conDf]).to_csv(pilot_file_loc + 'cleaned_data/cleaned12jan_p2.csv')